# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from scipy.stats.mstats import gmean

from sklearn.preprocessing import StandardScaler

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('df', engine)
X = df['message']
y = df.iloc[:,4:] #excludes ['message', 'genre', 'id', 'original']

In [3]:
df.shape

(26216, 40)

In [5]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y.shape

(26216, 36)

In [27]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df.iloc[0,2]

'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]


In [30]:
# test out funciton
for message in X[:3]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
# knn = KNeighborsClassifier(n_neighbors=3)
# classifier = MultiOutputClassifier(knn, n_jobs=-1)

In [32]:
forest_clf = RandomForestClassifier(n_estimators=10)

In [33]:
pipe = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [35]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
y_pred = pipe.predict(X_test)

In [37]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.62      0.37      0.47      1476
          1       0.83      0.93      0.88      5025
          2       0.65      0.21      0.31        53

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5368
          1       0.82      0.37      0.51      1186

avg / total       0.87      0.87      0.85      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3797
          1       0.75      0.51      0.61      2757

avg / total       0.73      0.72      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
# overall_accuracy = (y_pred == y_test).mean().mean()

# print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.15% 



In [39]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [41]:
multi_f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.15% 

F1 score (custom definition) 92.24%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [42]:
pipe.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4aea688620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None

In [44]:
# specify parameters for grid search
parameters = {
   'tfidf__use_idf':(True,False),
              'clf__estimator__min_samples_split': [2, 4],
              'clf__estimator__n_estimators': [50, 100]
              }
## RUNS IN 108 min
# Best parameters =  
#{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True}

In [45]:
# create grid search object
cv = GridSearchCV(pipe, param_grid=parameters, verbose =10, n_jobs=-1)
# verbose=1 or 2 to the fit function at least you'll know what's going on while it's training
# n_jobs is what controls paralllelism, if you set it to 4 it will fit models in your grid 4 by 4. 
## If you set it to -1 it will check how many cores your CPU has and use that number

In [46]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.22978333841928594, total= 3.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.23375038144644492, total= 3.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  7.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.23176685993286542, total= 3.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 10.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.23527616722612146, total= 3.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 17.6min remaining:    0.0s


In [ ]:
y_pred = cv.predict(X_test)

In [61]:
def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [ ]:
perf_report(cv, X_test, y_test)

In [29]:
print(cv.best_score_)

0.237870003052


In [30]:
best_parameters= cv.best_params_
print("Best parameters = ", best_parameters)

Best parameters =  {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True}


In [32]:
Best_parameters =  {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True}

In [27]:
# def load_data():
#     engine = create_engine('sqlite:///InsertDatabaseName.db')
#     df = pd.read_sql_table('df', engine)
#     X = df['message']
#     y = df.iloc[:,4:] #excludes ['message', 'genre', 'id', 'original']
#     return X, y


# def display_results(cv, y_test, y_pred):
#     labels = np.unique(y_pred)
#     confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
#     accuracy = (y_pred == y_test).mean()

#     print("Labels:", labels)
#     print("Confusion Matrix:\n", confusion_mat)
#     print("Accuracy:", accuracy)
#     print("\nBest Parameters:", cv.best_params_)


# def main():
#     X, y = load_data()
#     X_train, X_test, y_train, y_test = train_test_split(X, y)

#     model = pipe
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
    
#     display_results(model, y_test, y_pred)


# main()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
Best_parameters

NameError: name 'Best_parameters' is not defined

1) AdaBoost classifier instead of RandomForest

2) New feaure: case_normalizer

3) Overall accuracy: from 94.15% to 94.71%

4) f1-score: from 92.24% to 93.57%

In [52]:
from sklearn.base import BaseEstimator, TransformerMixin

class CaseNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.Series(X).apply(lambda x: x.lower()).values

case_normalizer = CaseNormalizer()

In [53]:
pipe2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('case_normalizer', CaseNormalizer())
        ])),

        #('clf', MultiOutputClassifier(AdaBoostClassifier()))
        ('clf',AdaBoostClassifier())
    ])

## results in error blocks[0,:] has incompatible row dimensions when trying to fit

In [55]:
pipe2 = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                    ])

In [56]:
X_train.shape

(19662,)

In [57]:
y_train.shape

(19662, 36)

In [58]:
pipe2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [59]:
y_pred2 = pipe2.predict(X_test)

In [62]:
perf_report(pipe2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.54      0.21      0.31      1476
          1       0.80      0.94      0.87      5025
          2       0.38      0.17      0.23        53

avg / total       0.74      0.77      0.73      6554

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5368
          1       0.76      0.54      0.64      1186

avg / total       0.88      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3797
          1       0.77      0.58      0.66      2757

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-sco

In [63]:

multi_f1 = multioutput_fscore(y_test,y_pred2, beta = 1)
overall_accuracy = (y_pred2 == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.71% 

F1 score (custom definition) 93.57%



### 9. Export your model as a pickle file

In [67]:
import pickle

In [68]:
filename = 'finalized_model.sav'
pickle.dump(pipe2, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.